In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_data'
table_conca = f'{project}.{dataset}.{table}'

schema = [
    {'name': 'Date', 'type': 'DATE'},
    {'name': 'Ticker', 'type': 'STRING'},
    {'name': 'Adj Close', 'type': 'INTEGER'},
    {'name': 'Close', 'type': 'INTEGER'},
    {'name': 'High', 'type': 'INTEGER'},
    {'name': 'Low', 'type': 'INTEGER'},
    {'name': 'Open', 'type': 'INTEGER'},
    {'name': 'Volume', 'type': 'INTEGER'},
]

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [5]:
unique_tickers = bigquery.run_query(
    f"""
    SELECT
        Symbol
    FROM sara-carles-keepcoding.bronze.bronze_sp500_tickers
    """
)
unique_tickers

,Symbol
0,COP
1,EOG
2,OXY
3,HES
4,FANG
...,...
498,TTWO
499,GOOGL
500,GOOG
501,META


In [6]:
# Asginamos una fecha inicial a cada ticker.
# En caso de no tenerlo ya en Biquery, nos traeremos los datos desde esta fecha
unique_tickers_initial_date = initial_date_by_ticker(unique_tickers, initial_date='2015-01-01')

In [7]:
try:
    max_date_by_ticker = last_date_by_ticker_saved(unique_tickers_initial_date, bigquery, table_conca)
    print(max_date_by_ticker)

except Exception as e:
    print(e)
    max_date_by_ticker = unique_tickers_initial_date
    print(max_date_by_ticker)    

'ticker'
    Symbol initial_date
0      COP   2015-01-01
1      EOG   2015-01-01
2      OXY   2015-01-01
3      HES   2015-01-01
4     FANG   2015-01-01
..     ...          ...
498   TTWO   2015-01-01
499  GOOGL   2015-01-01
500   GOOG   2015-01-01
501   META   2015-01-01
502   MTCH   2015-01-01

[503 rows x 2 columns]


In [ ]:
# Hacemos un incremental para traernos los últimos registros
df = fetch_historical_data(max_date_by_ticker, start_date_col='date', interval='1d')

Datos obtenidos para AAPL.
Datos obtenidos para AMGN.
Datos obtenidos para AMZN.
Datos obtenidos para AXP.
Datos obtenidos para BA.
Datos obtenidos para CAT.
Datos obtenidos para CRM.
Datos obtenidos para CSCO.
Datos obtenidos para CVX.
Datos obtenidos para DIS.
Datos obtenidos para DOW.
Datos obtenidos para GS.
Datos obtenidos para HD.
Datos obtenidos para HON.
Datos obtenidos para IBM.
Datos obtenidos para INTC.
Datos obtenidos para JNJ.
Datos obtenidos para JPM.
Datos obtenidos para KO.
Datos obtenidos para MCD.
Datos obtenidos para MMM.
Datos obtenidos para MRK.
Datos obtenidos para MSFT.
Datos obtenidos para NKE.
Datos obtenidos para PG.
Datos obtenidos para TRV.
Datos obtenidos para UNH.
Datos obtenidos para V.
Datos obtenidos para VZ.
Datos obtenidos para WMT.


In [ ]:
# Seleccionamos solo los resultados que aun no existen en Bigquery para no añadir duplicados
result = process_and_save_joins(df, max_date_by_ticker, bigquery, project, dataset, table)

In [7]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(result, project, dataset, table, if_exists='append')

NameError: name 'result' is not defined